This notebook is for evaluating a trained model.

## Imports

In [2]:
# Run this cell if you can't get the installation done in anaconda

!pip install rasterio wget tensorflow==2.4.1 tensorflow_addons[tensorflow]

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\anaconda3\\envs\\canopy_test_models\\lib\\site-packages\\~ensorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.




  Using cached tensorflow-2.4.1-cp37-cp37m-win_amd64.whl (370.7 MB)
  Using cached tensorflow_estimator-2.4.0-py2.py3-none-any.whl (462 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached grpcio-1.32.0-cp37-cp37m-win_amd64.whl (2.5 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=73d8721e619990120932d49123c9b4d9a5d28a84ba6cca5bbca68cdb0b3d04ca
  Stored in directory: c:\users\david\appdata\local\pip\cache\wheels\a1\b6\7c\0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.36.1
    Uninstalling grpcio-1.36.1:
      Successfully uninstalled grpcio-1.36.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Attempting uninst

In [3]:
import os
import multiprocessing
import rasterio
import tensorflow as tf
import tensorflow_hub as hub
from glob import glob
import pickle
import numpy as np
import os
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras import layers
from tensorflow_addons.metrics import F1Score
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy

from tensorflow import keras
import pandas as pd
#import matplotlib.pyplot as plt
import random
import time
import boto3
import io
from test_generator import TestGenerator
np.set_printoptions(suppress=True)
np.set_printoptions(precision=6)

In [2]:
print(tf.__version__)

2.4.1


# Updated Eval Code - To be used ad hoc for new subsequent checkpoints

In [2]:
### The label_list variable should be changed to whichever labels your model is actually using. ###
### You cna see examples of other label lists that were commented out.

# label_list = ["Habitation", "Industrial_agriculture", "Mining", "Rainforest", "River", "Roads", "Savannah", 
#               "Shifting_cultivation", "Water"]

#new_label_list = ["Industrial_agriculture","ISL","Mining","Roads","Shifting_cultivation"]
new_label_list = ['SAB', 'null']

# 1:"Industrial_agriculture"
# 2:"ISL"
# 3:"Mining"
# 4:"Roads"
# 5:"Shifting_cultivation"


numclasses = len(new_label_list)
# Use whichever "bands" variable corresponds to the bands the model actually trained on.
bands_all = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 17, 18]
bands_RGBNIRN_old = [2, 3, 4, 8,18]
bands_RGBNIRN = [2, 3, 4, 8,12,13]
bands_RGBNIRSWR1SWR2 = [2, 3, 4, 8,11,12]
bands_RGBNIRSWR1SWR2NDVI = [2, 3, 4, 8,11,12,18]
# input_shape_all_bands= (100,100,len(bands_all))
input_shape_RGBNIRN = (100,100,len(bands_RGBNIRN))
input_shape_RGBNIRN_old = (100,100,len(bands_RGBNIRN_old))
input_shape_RGBNIRSWR1SWR2 = (100,100,len(bands_RGBNIRSWR1SWR2))
input_shape_RGBNIRSWR1SWR2NDVI = (100,100,len(bands_RGBNIRSWR1SWR2NDVI))
input_shape_all = (100,100,len(bands_all))

margaux_LR = 0.001


In [5]:
### The define_model function needs to be **the exact same** as the one used for training ###

def define_model(numclasses, xbands_input_shape, output_type, starting_checkpoint=None, lcl_chkpt_dir=None,s3_chkpt_dir=None):

    print(f"Using Pre-trained {pre_trained_model} model")

    # parameters for CNN
    input_tensor = Input(shape=xbands_input_shape, name='input_x')

    # introduce a additional layer to get from bands to 3 input channels
    input_tensor = Conv2D(3, (1, 1))(input_tensor)

    if pre_trained_model == "resnet50":

        base_model_pre_trained = keras.applications.ResNet50(include_top=False,
                                                          weights='imagenet',
                                                          input_shape=(100, 100, 3))
        base_model = keras.applications.ResNet50(include_top=False,
                                                 weights=None,
                                                 input_tensor=input_tensor)

    if pre_trained_model == "resnet_sentinel":

        base_model_pre_trained = tf.keras.Sequential([
            hub.KerasLayer('sentinel_resnet_tf')
        ])

        base_model = keras.applications.ResNet50(include_top=False,
                                                 weights=None,
                                                 input_tensor=input_tensor)

        # RGB_base_model = keras.applications.ResNet50(include_top=False,
        #                                                   weights='imagenet',
        #                                                   input_shape=(100, 100, 3))

        # xbands_base_model = keras.applications.ResNet50(include_top=False,
        #                                          weights=None,
        #                                          input_tensor=input_tensor)  #Ideally, this would be a resnet50 trained on the extra bands

        # RGB_base_model.layers[0]._name = 'input_RGB'

        # for layer in xbands_base_model.layers :
        #     layer._name = layer.name + str('_x')

        # premerge_RGB_model =  Model(inputs=RGB_base_model.input, outputs=RGB_base_model.get_layer('conv2_block3_out').output)
        # premerge_xbands_model = Model(inputs=xbands_base_model.input, outputs=xbands_base_model.get_layer('conv2_block3_out_x').output)
        # merged_features = Add()([premerge_RGB_model.output, premerge_xbands_model.output])
        # premerge_model = Model(inputs= [premerge_RGB_model.input, premerge_xbands_model.input],outputs = merged_features)
        # postmerge_model = Model(inputs = xbands_base_model.get_layer('conv3_block1_1_conv_x').input, outputs= xbands_base_model.output)
        # full_output = postmerge_model(premerge_model.output)
        # full_model = Model(inputs= [premerge_RGB_model.input, premerge_xbands_model.input], outputs= full_output)

    if pre_trained_model == "efficientnetb0":

        base_model_pre_trained = keras.applications.EfficientNetB0(include_top=False,
                                                      weights="imagenet",
                                                      input_shape=(100, 100, 3))

        base_model = keras.applications.EfficientNetB0(include_top=False,
                                     weights=None,
                                     input_tensor=input_tensor)



    for i, layer in enumerate(base_model_pre_trained.layers):
        # we must skip input layer, which has no weights
        if i == 0:
            continue
        base_model.layers[i + 1].set_weights(layer.get_weights())

    # add a global spatial average pooling layer
    top_model = base_model.output
    # top_model = full_model.output
    top_model = GlobalAveragePooling2D()(top_model)

    # let's add a fully-connected layer
    top_model = Dense(2048, activation='relu')(top_model)
    top_model = Dense(2048, activation='relu')(top_model)
    # and a logistic layer
    predictions = Dense(numclasses, activation='softmax')(top_model)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    # model = Model(inputs=full_model.input, outputs=predictions)
#         model.summary()
#         last_chkpt_path = lcl_chkpt_dir + 'last_chkpt.h5'

    start_epoch = 0 
    chkpt_name = 'model_SAB_newweights.h5'
    model.load_weights(chkpt_name)

    return model,start_epoch

In [6]:
numclasses

2

In [7]:
input_shape_RGBNIRSWR1SWR2NDVI

(100, 100, 7)

In [8]:
# Make sure all variables are the same as the ones used for training
pre_trained_model = "resnet_sentinel"
model_RGBNIRN, start_epoch = define_model(2, (100,100,6), 'multiclass')

Using Pre-trained resnet_sentinel model


In [9]:
# Set the metrics we want to use for the evaluation
model_RGBNIRN.compile(loss=BinaryCrossentropy(), # https://www.tensorflow.org/addons/api_docs/python/tfa/losses/SigmoidFocalCrossEntropy
                  optimizer=keras.optimizers.Adam(margaux_LR),
                   metrics=[tf.metrics.BinaryAccuracy(name='accuracy'),
                           tf.keras.metrics.Precision(name='precision'),
                           tf.keras.metrics.Precision(class_id=0,name='SAB_precision'),
                           # Computes the precision of the predictions with respect to the labels.
                           tf.keras.metrics.Recall(name='recall'),
                           tf.keras.metrics.Recall(class_id=0,name='SAB_recall'),
                           # Computes the recall of the predictions with respect to the labels.
                           F1Score(num_classes=numclasses, name="f1_score")
                           # https://www.tensorflow.org/addons/api_docs/python/tfa/metrics/F1Score
                           ]
                  )




In [7]:
### This cell downloads the final model. You can also just download it via the S3 console if you want. ###

# s3_uri = "s3://canopy-production-ml-output/ckpt/pc-tf-custom-container-2021-04-20-05-16-47-844/model_weights_very_best.h5"
s3_uri = 's3://canopy-production-ml-output/ckpt/SAB_only_100_epochs_1_814/last_chkpt.h5'

KEY = "/".join(s3_uri.split("/")[3:])
s3 = boto3.resource('s3')
s3.Bucket('canopy-production-ml-output').download_file(KEY, 'model_weights_SAB.h5')

In [13]:
### Load model weights ###

# model_all_bands.load_weights("model_weights_all_bands.h5")
model_RGBNIRN.load_weights("model_SAB_newweights_2.h5")

# Eval Pipeline - To be used once checkpoint is finalized

In [30]:
# Make a new label json file for the SAB-only model

import json
sab_labels = {'label_names': {0: "SAB", 1: "null"}}
with open('SAB_labels.json', 'w') as f:
    json.dump(sab_labels, f)

In [28]:
bands_RGBNIRSWR1SWR2NDVI

[2, 3, 4, 8, 11, 12, 18]

In [12]:
batch_size = 64
train_set = "DRC_labels_SAB_train_v1.csv" # location of the train labels CSV
test_set = "DRC_labels_SAB_test_v1.csv" # location of the test labels CSV
val_labels = "DRC_labels_SAB_val_v1.csv" # Location of the val labels CSV

gen_RGBNIRN = TestGenerator(label_file_path_test=test_set,
                            file_mode='file', # "File" is used because the test data is on a local machine
                            training_dir = "D:/canopy_data/s2cloudless_new_model/", # location of the test images
                             #bucket_name='canopy-production-ml', # this argument is only used when file_mode is "s3"
                             label_mapping_path="SAB_labels.json", # the location of the labels json
                             data_extension_type='.tif',
                             bands=[2, 3, 4, 8, 11, 12],
                             test_data_shape=(100,100,6),
                             test_data_batch_size=batch_size,
                             enable_data_prefetch=False,
                             data_prefetch_size=tf.data.experimental.AUTOTUNE,
                             num_parallel_calls=2 * multiprocessing.cpu_count(),
                             output_shape=(tf.float32, tf.float32))

The file DRC_labels_SAB_test_v1.csv is missing positive labels for classes []
test on 3964 images


In [13]:
model_RGBNIRN.evaluate(gen_RGBNIRN.test_dataset)

# Might want to look into rewriting that line in TestGenerator to get rid of the warning

C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all 

C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all 

C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all 

 1/62 [..............................] - ETA: 7:32 - loss: 0.7548 - accuracy: 0.9375 - precision: 0.9375 - SAB_precision: 0.7143 - recall: 0.9375 - SAB_recall: 0.7143 - f1_score: 0.8396

C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all 

...

C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all 

C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all 

C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  label = self.labels_file_test.drop('paths', 1).iloc[int(id)].to_list()
C:\Users\David\canopy\cb_feature_detection\model-development\test_generator.py:201: FutureWarning: In a future version of pandas all 

62/62 [==============================] - 271s 4s/step - loss: 1.0924 - accuracy: 0.8943 - precision: 0.8943 - SAB_precision: 0.6919 - recall: 0.8943 - SAB_recall: 0.6442 - f1_score: 0.8022


[1.0924255847930908,
 0.8942986726760864,
 0.8942986726760864,
 0.6919274926185608,
 0.8942986726760864,
 0.6441717743873596,
 array([0.667196, 0.937172], dtype=float32)]

In [14]:
# SAB precision of 69.19% and recall of 64.42% is the best I got

model_RGBNIRN.save('best_SAB_model.h5')